In [1]:
import numpy as np
import pandas as pd
import MyLogisticRegression as lr

In [2]:
data = pd.read_csv('bank.csv', sep = ',')

data = data.drop(data[data.job == 'unknown'].index)
data = data.drop(data[data.education == 'unknown'].index)
data = data.drop(columns=['contact', 'day', 'month'])

data = data.rename(columns={'y':'deposit'})
data.insert(len(data.columns), 'y', data.default)
data = data.drop(columns=['default'])

print(data.shape)
data.head()

(4311, 14)


,age,job,marital,education,balance,housing,loan,duration,campaign,pdays,previous,poutcome,deposit,y
0,30,unemployed,married,primary,1787,no,no,79,1,-1,0,unknown,no,no
1,33,services,married,secondary,4789,yes,yes,220,1,339,4,failure,no,no
2,35,management,single,tertiary,1350,yes,no,185,1,330,1,failure,no,no
3,30,management,married,tertiary,1476,yes,yes,199,4,-1,0,unknown,no,no
4,59,blue-collar,married,secondary,0,yes,no,226,1,-1,0,unknown,no,no


In [3]:
job_val = {'unemployed':0, 
           'student':1,
           'retired':2,
           'services':3, 
           'housemaid':4, 
           'blue-collar':5,
           'self-employed':6,
           'technician':7,
           'admin.':8, 
           'management':9, 
           'entrepreneur':10
          }

marital_val = {'divorced':-1,
               'single':0,
               'married':1}

boolean_val = {'yes':1,
               'no':0}

education_val = {'primary':1,
                 'secondary':2,
                 'tertiary':3}

poutcome_val = {'unknown':0,
                'failure':-1,
                'other':0,
                'success':1}

data['job'] = np.array([job_val[x] for x in data['job']], dtype = 'int')
data['marital'] = np.array([marital_val[x] for x in data['marital']], dtype = 'int')
data['education'] = np.array([education_val[x] for x in data['education']], dtype = 'int')
data['poutcome'] = np.array([poutcome_val[x] for x in data['poutcome']], dtype = 'int')

for token in ['housing', 'loan', 'deposit', 'y']:
    data[token] = np.array([boolean_val[x] for x in data[token]], dtype = 'int')

print(data.dtypes)
data

age          int64
job          int32
marital      int32
education    int32
balance      int64
housing      int32
loan         int32
duration     int64
campaign     int64
pdays        int64
previous     int64
poutcome     int32
deposit      int32
y            int32
dtype: object


,age,job,marital,education,balance,housing,loan,duration,campaign,pdays,previous,poutcome,deposit,y
0,30,0,1,1,1787,0,0,79,1,-1,0,0,0,0
1,33,3,1,2,4789,1,1,220,1,339,4,-1,0,0
2,35,9,0,3,1350,1,0,185,1,330,1,-1,0,0
3,30,9,1,3,1476,1,1,199,4,-1,0,0,0,0
4,59,5,1,2,0,1,0,226,1,-1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4516,33,3,1,2,-333,1,0,329,5,-1,0,0,0,0
4517,57,6,1,3,-3313,1,1,153,1,-1,0,0,0,1
4518,57,7,1,2,295,0,0,151,11,-1,0,0,0,0
4519,28,5,1,2,1137,0,0,129,4,211,3,0,0,0


In [4]:
print(data[data['y'] == 1].shape)
data[data['y'] == 1]

(73, 14)


,age,job,marital,education,balance,housing,loan,duration,campaign,pdays,previous,poutcome,deposit,y
48,32,10,0,1,-849,1,1,204,1,-1,0,0,0,1
71,42,10,-1,3,2,1,0,380,1,-1,0,0,0,1
124,51,9,0,3,-55,1,0,281,2,266,6,-1,0,1
152,45,0,-1,1,-249,1,1,92,1,-1,0,0,0,1
214,31,9,1,3,-253,0,0,220,4,-1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4089,32,5,1,2,-53,1,0,648,1,272,1,0,1,1
4120,31,6,0,3,-282,1,0,131,3,-1,0,0,0,1
4257,32,3,1,2,-220,1,1,123,2,-1,0,0,0,1
4317,27,3,0,2,-1,0,0,60,2,-1,0,0,0,1


In [5]:
l = lr.LogisticRegressionHelper()

true_set = data[data['y'] == 1].values
false_set = data[data['y'] == 0].values

training_set = np.r_[true_set[:50, :], false_set[:50, :]]
test_set = np.r_[true_set[50:70, :], false_set[80:100, :]]

l.SetTrainDataSet(training_set[:,:-1], training_set[:,-1:])
l.SetTestDataSet(test_set[:,:-1], test_set[:,-1:])

l.Training()

print('J = {}'.format(l.J_list))
print('Theta = {}'.format(l.theta))
print('Accuracy = {}'.format(l.GetAccuracy()))
print('F1 P R = {}'.format(l.GetFPR()))

The row number of _X_train is different to the length of theta, so theta has been reset.
WARNING! Theta is not well shaped, so it has been reset.
J = [0.69314718 0.69314718]
Theta = [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
Accuracy = 0.5


ZeroDivisionError: division by zero